## Generate relevant plots from the fix reports after running the tests

In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def parse_report(path: Path) -> dict:
    metrics = {}
    num_pat = re.compile(r"[-+]?\d*\.?\d+")
    with path.open(encoding="utf-8") as fh:
        for line in fh:
            if ":" not in line:
                continue
            label, _ = line.split(":", 1)
            label = label.strip()
            m = num_pat.search(line)
            if m:
                metrics[label] = float(m.group())

    required = [
        "Total Instances", 
        "Total Correct",
        "Total Runtime Failed", 
        "Total Compilation Failed",
        "Total Test Failed", 
        "Total Infinite Loop",
        "Accuracy", 
        "Runtime Rate", 
        "Compilation Rate",
        "Test Failed Rate", 
        "Infinite Loop Rate"
    ]

    missing = [k for k in required if k not in metrics]
    if missing:
        raise ValueError(f"{path.name}: missing fields {missing}")

    return metrics

## Mapping generated fix report files

In [7]:
files = [
    ("fix_reports/openai_gpt_4o_avatar_compileReport_from_Python_to_Java.txt", "Avatar - Python → Java"),
    ("fix_reports/openai_gpt_4o_codenet_compileReport_from_Python_to_Java.txt", "Codenet - Python → Java"),
    ("fix_reports/openai_gpt_4o_avatar_compileReport_from_Java_to_Python.txt", "Avatar - Java → Python"),
    ("fix_reports/openai_gpt_4o_codenet_compileReport_from_Java_to_Python.txt", "Codenet - Java → Python"),
]

In [8]:
records, experiments = {}, []
for fname, label in files:
    records[label] = parse_report(Path(fname))
    experiments.append(label)

total_instances = [records[l]["Total Instances"]          for l in experiments]
total_correct   = [records[l]["Total Correct"]            for l in experiments]
runtime_failed  = [records[l]["Total Runtime Failed"]     for l in experiments]
comp_failed     = [records[l]["Total Compilation Failed"] for l in experiments]
test_failed     = [records[l]["Total Test Failed"]        for l in experiments]
infinite_loop   = [records[l]["Total Infinite Loop"]      for l in experiments]

accuracy_rate   = [records[l]["Accuracy"]             for l in experiments]
runtime_rate    = [records[l]["Runtime Rate"]         for l in experiments]
comp_rate       = [records[l]["Compilation Rate"]     for l in experiments]
test_rate       = [records[l]["Test Failed Rate"]     for l in experiments]
loop_rate       = [records[l]["Infinite Loop Rate"]   for l in experiments]

fail_counts     = [i - c for i, c in zip(total_instances, total_correct)]

## Loading experiment data into dataframe

In [9]:
df = pd.DataFrame({
    "Total Inst": total_instances,
    "Correct": total_correct,
    "Runtime Fail": runtime_failed,
    "Compile Fail": comp_failed,
    "Test Fail": test_failed,
    "Inf Loop": infinite_loop,
    "Accuracy (%)": np.round(accuracy_rate, 2),
    "Runtime Rate (%)": np.round(runtime_rate, 2),
    "Compile Rate (%)": np.round(comp_rate, 2),
    "Test Rate (%)": np.round(test_rate, 2),
    "Loop Rate (%)": np.round(loop_rate, 2)
    }, index=experiments
)